In [2]:
import redis
import json

# Define connection variables
host = 'localhost'
port =  6379
password = None #'<password>'

# Connect to Redis
r = redis.Redis(host=host, port=port, password=password, decode_responses=True)
print('Connected to Redis')

r.flushdb()

Connected to Redis


True

# Data Types

## Description of the use case
This notebook introduces Redis data types.

## Redis Data Structures Used

- **Geospatial**: Geo operations.
- **TimeSeries**: Time Series Operations
- **HyperLogLog**: HyperLogLog Operations (Probabilitic - Cardinality)
- **BloomFilter/CuckooFilter**: BloomFilter/CuckooFilter Operations (Probabilitic - Existence)
- **JSON**: The JSON type support


## Geospatial
Redis geospatial data-type stores coordinates and search them. In this example we find the nearby vendors (based on current location). This is useful in finding nearby vendors offers for the card or the app customers.

In [12]:
r.geoadd("vendors", [28.320491, 37.805186, '{"name":"Starbucks", "type":"cafe"}'])
r.geoadd("vendors", [28.322751,37.485508, '{"name":"Westside", "type":"clothing"}'])
r.geoadd("vendors", [28.361479,37.48578, '{"name":"Costa Cafe", "type":"cafe"}'])
r.geoadd("vendors", [28.351032,37.48621, '{"name":"Reliance Mart", "type":"retail"}'])
r.geoadd("vendors", [28.346185,37.489204, '{"name":"Chroma", "type":"electronics"}'])

res4 = r.geosearch(
    "vendors",
    longitude=28.359654,
    latitude=37.486465,
    radius=2000,
    unit="m",
    withdist=True
)
print(res4)  # >>> ['station:1', 'station:2', 'station:3']



[['{"name":"Chroma", "type":"electronics"}', 1227.3095], ['{"name":"Reliance Mart", "type":"retail"}', 761.4476], ['{"name":"Costa Cafe", "type":"cafe"}', 178.0393]]


## Time Series
The Redis time series structure stores and allow query on timestamped data points.

In [14]:
# Use TimeSeries to store historical sales data
r.ts().add('sales:newaccounts', '*', 100) # add with current timestamp
last_sale = r.ts().get('sales:newaccounts')
print(last_sale)
r.ts().incrby('sales:newaccounts', 1, timestamp='*')
last_sale = r.ts().get('sales:newaccounts')
print(last_sale)
r.ts().incrby('sales:newaccounts', 4, timestamp='*')

last_sale = r.ts().get('sales:newaccounts')
print(last_sale)
# Get a range of data points
range_data = r.ts().range('sales:newaccounts', '0', '+')
print(f"Range Data: {range_data}")
aggregated_data = r.ts().range('sales:newaccounts', '-', '+', aggregation_type='avg', bucket_size_msec=60000)
print(f"Aggregated Data: {aggregated_data}")

(1749563519668, 100.0)
(1749563519669, 101.0)
(1749563519670, 105.0)
Range Data: [(1749563403890, 100.0), (1749563403893, 105.0), (1749563437875, 100.0), (1749563437877, 101.0), (1749563437878, 105.0), (1749563454420, 100.0), (1749563454422, 101.0), (1749563454423, 105.0), (1749563475155, 100.0), (1749563475157, 101.0), (1749563475158, 105.0), (1749563479151, 100.0), (1749563479154, 101.0), (1749563479155, 105.0), (1749563485496, 100.0), (1749563485498, 101.0), (1749563485499, 105.0), (1749563507341, 100.0), (1749563507343, 101.0), (1749563507344, 105.0), (1749563519668, 100.0), (1749563519669, 101.0), (1749563519670, 105.0)]
Aggregated Data: [(1749563400000, 102.125), (1749563460000, 102.0)]


## HyperLogLog
Redis HyperLogLog provides an efficient way to estimate the cardinality (number of unique elements) of a large set without storing the entire set.

In [17]:
# add items to a hyper log
r.pfadd("feature:newUI", "user:1")
r.pfadd("feature:newUI", "user:5")
r.pfadd("feature:newUI", "user:13")
r.pfadd("feature:newUI", "user:52")
r.pfadd("feature:newUI", "user:31")
r.pfadd("feature:newUI", "user:5")
r.pfadd("feature:newUI", "user:5")
r.pfadd("feature:newUI", "user:52")
r.pfcount("feature:newUI")


5

## BloomFilter
A Bloom filter is a probabilistic data structure in Redis that enables to check if an element is present in a set using a very small memory space of a fixed size. 


In [9]:
r.flushdb()
r.bf().reserve("blacklist_ips", 0.01, 1000)
r.bf().madd("blacklist_ips",'193.176.23.7','212.18.104.18','213.55.85.202','27.254.235.3','34.66.72.251','45.148.10.240','45.148.10.79',
                  '51.254.101.166','59.12.160.91','80.242.208.68','88.214.48.16','88.214.48.17','88.214.48.18','91.205.219.185')
is_blacklisted = r.bf().exists("blacklist_ips","59.12.160.91")
print(is_blacklisted)
is_blacklisted = r.bf().exists("blacklist_ips","127.0.0.1")
print(is_blacklisted)


1
0


## CuckooFilter
A Cuckoo filter, just like a Bloom filter, is a probabilistic data structure in Redis that checks if an element is present in a set in a very fast and space efficient way, while also allowing for deletions

In [13]:
r.flushdb()
# try ip filtering with cuckoo filter
r.cf().insert("blacklist_ips_cf", items=['193.176.23.7','212.18.104.18','213.55.85.202','27.254.235.3','34.66.72.251','45.148.10.240','45.148.10.79',
                  '51.254.101.166','59.12.160.91','80.242.208.68','88.214.48.16','88.214.48.17','88.214.48.18','91.205.219.185'])
is_blacklisted = r.cf().exists("blacklist_ips_cf","59.12.160.91")
print(is_blacklisted)
is_blacklisted = r.cf().exists("blacklist_ips_cf","127.0.0.1")
print(is_blacklisted)
r.cf().delete("blacklist_ips_cf","59.12.160.91");
is_blacklisted = r.cf().exists("blacklist_ips_cf","59.12.160.91")
print(is_blacklisted)


1
0
0


## JSON
The JSON capability of Redis Open Source provides JavaScript Object Notation (JSON) support for Redis. It lets you store, update, and retrieve JSON values in a Redis database,

In [45]:
r.json().set("session:u00001", "$", {"user_id": "u00001", "roles": ["auth-user", "sa-user", "fd-user"], "recent_views": ["product:2001", "product:2002"]})
print(r.json().get("session:u00001", "$"))
r.json().merge("session:u00001", "$", {"jwt":"eyckskjdjkdvkr7bfb35nb3bnhrg34jh43"}) # the new attribute jwt
print(r.json().get("session:u00001", "$"))

r.json().arrappend("session:u00001", "$.recent_views", "product:9999") # the existing array appened.
print(r.json().get("session:u00001", "$"))

r.json().merge("session:u00001", "$", {"recent_views":None}) # remove existing attribute
print(r.json().get("session:u00001", "$"))

[{'user_id': 'u00001', 'roles': ['auth-user', 'sa-user', 'fd-user'], 'recent_views': ['product:2001', 'product:2002']}]
[{'user_id': 'u00001', 'roles': ['auth-user', 'sa-user', 'fd-user'], 'recent_views': ['product:2001', 'product:2002'], 'jwt': 'eyckskjdjkdvkr7bfb35nb3bnhrg34jh43'}]
[{'user_id': 'u00001', 'roles': ['auth-user', 'sa-user', 'fd-user'], 'recent_views': ['product:2001', 'product:2002', 'product:9999'], 'jwt': 'eyckskjdjkdvkr7bfb35nb3bnhrg34jh43'}]
[{'user_id': 'u00001', 'roles': ['auth-user', 'sa-user', 'fd-user'], 'jwt': 'eyckskjdjkdvkr7bfb35nb3bnhrg34jh43'}]
